<a href="https://colab.research.google.com/github/FatemehKV/binaryclassification/blob/main/fakeVsvalidnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Embedding

In [ ]:
df = pd.read_csv('/home/train.csv',error_bad_lines=False, engine="python",encoding='latin-1')
df_test = pd.read_csv('/home/test.csv',error_bad_lines=False, engine="python",encoding='latin-1')
df_test_labels = pd.read_csv('/home/labels.csv',error_bad_lines=False, engine="python",encoding='latin-1')
#Merging test data and labels in one file
res = pd.merge(df_test, df_test_labels, on='id')

In [ ]:
#Custom cleaning pipeline
!pip install texthero
#texthero needs spacy as a requirement
!python -m spacy validate
!python -m spacy download en_core_web_sm
import texthero as hero
from texthero import preprocessing


In [16]:

custom_pipeline = [preprocessing.fillna,
                   preprocessing.lowercase,
                   preprocessing.remove_whitespace,
                   preprocessing.remove_diacritics, preprocessing.remove_punctuation,preprocessing.remove_digits,
                   preprocessing.remove_urls,preprocessing.remove_stopwords
                  ]
#Preprocessing the training data
df['clean_text'] = hero.clean(df['text'], custom_pipeline)
df= df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
#Before proceeding, we need to check whether a null value is present in our dataset or not.
df.dropna(inplace=True)
df.drop(df[(df['clean_text'] == ' ')].index, inplace=True)
df.drop(df[(df['label'] != '0') & (df['label']!='1')].index, inplace=True)



In [19]:
#Preprocessing the test data
res['clean_text'] = hero.clean(res['text'], custom_pipeline)
res= res.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
res.dropna(inplace=True)


In [21]:
max_len=2000
max_words = 400
X_train=df['clean_text']
token = Tokenizer(num_words=max_words,lower=True, split=' ')
token.fit_on_texts(X_train.values)
sequences = token.texts_to_sequences(X_train.values)
train_sequences_padded = sequence.pad_sequences(sequences, maxlen=max_len,padding='post')


In [ ]:
embed_dim = 64
lstm_out = 64
batch_size = 32
model = Sequential()
model.add(Embedding(max_words, embed_dim, input_length = max_len))
model.add(LSTM(lstm_out))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1, name='out_layer'))
model.add(Activation('sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',\
               metrics = ['accuracy'])
print(model.summary())

In [ ]:
y_train = df['label'].values.astype(int)
history=model.fit(train_sequences_padded, y_train, batch_size=batch_size, epochs = 10)

Epoch 1/10
127/572 [=====>........................] - ETA: 7:25 - loss: 0.6881 - accuracy: 0.5546

In [ ]:
X_test=res['clean_text']
sequences1=token.texts_to_sequences(X_test.values)
test_sequences_padded = sequence.pad_sequences(sequences1,maxlen=max_len)
y_test=res['label'].astype(int)
scores = model.evaluate(test_sequences_padded, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))